## Aggregation: valores estatísticos
---

In [1]:
import pandas as pd
import numpy as np

*aggregation* são valores estatístico de um conjunto de dados, como média, mediana, soma de todos os valores, etc. Pandas trabalha com os mesmo aggragators do numpy:
np.funções()|operação
---|---|
std|desvio padrão|
var|variância|
median|mediana|
percentile|porcentagem|
any|or|
all|and|

para series, os aggragators retornam um valor único:

In [2]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [3]:
ser.mean()

0.5623850983416314

In [4]:
ser.sum()

2.811925491708157

para datasets, por padrão, os aggregator retornam um valor para cada coluna:

In [5]:
df = pd.DataFrame({'A': rng.rand(5),'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [6]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

In [7]:
df.prod()

A    0.003340
B    0.000642
dtype: float64

especificando o eixo com `axis=`, é possível calcular para cada linha:

In [8]:
df.mean(axis=1)

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

In [9]:
df.prod(axis=1)

0    0.003211
1    0.056336
2    0.721042
3    0.127640
4    0.128745
dtype: float64

a função `describe()` apresenta os diversos resultados para aggregators para um dataset:

In [10]:
df.describe()

,A,B
count,5.000000,5.000000
mean,0.477888,0.443420
std,0.353125,0.426952
min,0.058084,0.020584
25%,0.155995,0.181825
50%,0.601115,0.212339
75%,0.708073,0.832443
max,0.866176,0.969910


In [11]:
ser.describe()

count    5.000000
mean     0.562385
std      0.308748
min      0.156019
25%      0.374540
50%      0.598658
75%      0.731994
max      0.950714
dtype: float64

outros aggregators próprios do pandas são:
pd.função()|operação
---|---|
count|total de itens
first|primeiro item
last|último item
mean|média
median|mediana
min|mínimo
max|máximo
std|desvio padrão
var|variância
mad|desvio médio absoluto

#### .groupby()
---

é uma forma de calcular os aggregates de um conjunto de informações através de um rótulo ou índice seguindo a ideia *split-apply-combine*:
1. o *split* involve a separação e o agrupamento de um dataframe dependendo do valor de uma chave passada pelo usuário;
2. o *apply* é a computação dos valores estatísticos;
3. o *combine* mescla os resultados e as apresentadas como um array.

In [12]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [13]:
dfg = df.groupby('key')
dfg

observe que é necessário passar o nome da coluna a qual deseja-se os cálculos, e o retorno da função é um objeto, não o resultado final. Isto é interessante porque, apenas quando, de fato, for solicitado o aggregate, é que seu resultado é apresentado:

In [14]:
dfg.sum()

,data
key,
A,3
B,5
C,7


In [15]:
dfg.prod()

,data
key,
A,0
B,4
C,10


In [16]:
dfg.count()

,data
key,
A,2
B,2
C,2


qualquer método de aggregation de pandas ou numpy é aceito pelo objeto `groupby`, bem como, qualquer operação suportada pelo series e dataframe.

inclusive, o objeto `groupby` pode receber indexing e slicing normalmente:

In [17]:
dfg_chave = dfg['key']
dfg_chave

e, da mesma forma, pode calcular os aggregators:

In [18]:
dfg_chave.sum()

key
A    AA
B    BB
C    CC
Name: key, dtype: object

objetos `groupby` também suportam iteração:

In [19]:
for i in dfg:
    print(i)

('A',   key  data
0   A     0
3   A     3)
('B',   key  data
1   B     1
4   B     4)
('C',   key  data
2   C     2
5   C     5)


é, inclusive, possível usar o describe junto do objeto `groupby`:

In [20]:
dfg.describe()

data                                         
    count mean      std  min   25%  50%   75%  max
key                                               
A     2.0  1.5  2.12132  0.0  0.75  1.5  2.25  3.0
B     2.0  2.5  2.12132  1.0  1.75  2.5  3.25  4.0
C     2.0  3.5  2.12132  2.0  2.75  3.5  4.25  5.0

sendo, este quadro, referente somente à coluna passada como parâmetro ao ser criado o objeto `groupby`.

o objeto `groupby` tem ainda uma variedade de métodos disponíveis que confere uma maior flexibilidade no cálculo das estatísticos do conjunto de dados.

o método `.aggregate()` pode recebe uma string, uma função, ou uma lista, por exemplo, e calcula todos os aggregators de uma só vez:

In [21]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],'data1': range(6),'data2': rng.randint(0, 10, 6)},columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [22]:
dfg = df.groupby('key')
dfg.aggregate(['min', 'median', 'max'])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

observe que as strings passadas em `.aggregate()` foram usadas para calculá-las em cada coluna.

se desejar especificar uma operação diferente para cada coluna, use dicionários:

In [23]:
dfg.aggregate({'data1': 'min', 'data2':'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


é possível filtrar os resultados usando o método `.filter()`, que recebe uma função que apresente algum critério de filtragem:

In [24]:
filter_func = lambda x: x['data2'].std() > 4
dfg.filter(filter_func)

,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


o método `.transform()` faz alguma tranformação que o usuário deseja no objeto `groupby` original. também é necessário criar uma função separada para isso, que o método `.transform()` recebe como parâmetro:

In [25]:
double = lambda x: x + x
dfg.transform(double)

,data1,data2
0,0,10
1,2,0
2,4,6
3,6,6
4,8,14
5,10,18


o método `.apply()` também faz transformação, só que nos níveis mais do `groupby` original, não necessitando transformar todo o objeto:

In [26]:
coloquially = lambda x: x['data1'] + x['data2']**2
dfg.apply(coloquially)

key   
A    0    25
     3    12
B    1     1
     4    53
C    2    11
     5    86
dtype: int64

o mpetodo `.apply()` sempre retorna ou um objeto panda ou um valor numérico.

é importante observar que `.groupby()` recebe mais do que só o nome de uma coluna. este pode receber listas, arrays, series, indexings, etc., para criar um grupo.

In [34]:
lista = [0, 1, 0, 1, 2, 0]
outro = df.groupby(lista)
outro.sum()

,key,data1,data2
0,ACC,7,17
1,BA,4,3
2,B,4,7


o mesmo exemplo `df.groupby('key')` poderia ser feito `df.groupby(df['key'])`:

In [39]:
df.groupby(df['key'])

um dicionário pode ser usado para mapear os índices para, por exemplo, classificá-los:

In [41]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
df2

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9


In [42]:
df2.groupby(mapping).sum()

,data1,data2
key,,
consonant,12,19
vowel,3,8


na verdade, `.groupby()` pode receber qualquer função do python:

In [43]:
df2.groupby(str.lower).sum()

,data1,data2
key,,
a,3,8
b,5,7
c,7,12


In [44]:
df2.groupby(str.lower).mean()

,data1,data2
key,,
a,1.5,4.0
b,2.5,3.5
c,3.5,6.0


neste exemplo, observe que, como foi passado `str.lower` como parâmetro para `.groupby()`, tudo poderia ter sido escrito em litras minúsculas foi modificado. depois, como foi solicitado a soma e a média, foi calculado tais valores.

através de uma lista, é possível passar quantas funções desejar para `.groupby()`:

In [45]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
key,key,,
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


sendo, desta forma, possível modificar o dataset inicial amplamente, sem modificar o dataset original.